# New topo DEMs in Southwest WA
## Some Observations and Questions

We have been investigating the new CUDEM tiles published in December, 2025 and have some questions...

This notebook illustrates some of the questions, and uses the list of topo sources found here:

https://depts.washington.edu/ptha/CHTuser/topo/topo-sources-cascadia/

This is a possibly incomplete list of places where we have found the topo DEMs for the Cascadia region. Corrections or additions welcome!

In many cases the most recent tiles are available in two different forms:

- A tif file referenced to NAVD88 e.g. in the catalog
  
  https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/index.html
  
- A netCDF file referenced to MHW (on the THREDDS server in folders for `nthmp`), e.g. in the catalog

  https://www.ngdc.noaa.gov/thredds/catalog/tiles/nthmp/tiled_19as/catalog.html

Many of our questions pertain to the difference between these:

1. Why are different formats (tif/nc) used for the different versions?
2. When reading these files, it seems that the `x,y` values in the .nc files are offset by integer multiples of $\Delta = $ 1/9" from integer values of longitude, latitude.  By contrast, the `x,y` in the .tif files are offset by half-integral offsets.  In the .nc files the first and last latitudes are exactly at 47.00 and 47.25, for example, whereas in the .tif files there are always a few points outside the tile with `y` values at $47 \pm \Delta/2$, for example.  Why are these indexed differently?
3. Does this mean that one set of values should be viewed as pointwise values and the other as cell average values?
4. How was the conversion done from .tif to .nc?  Which of these is considered more accurate, and the one that we should use if possible?

Note that the elevations `z` in the two files are not identical (even after accounting for the difference from NAVD88 to MHW), so they are not the same values simply offset by $\Delta/2$.

5. I assume the conversion from NAVD88 to MHW was done with VDATUM?  Was this applied to the entire .tif file? (The online version seems to only allow ASCII files or single points as input.)
6. In at least one case, `n47x00_w124x25`, the .nc file is labelled `2025v1` while the .tif file is labelled `2025v2`. Is the .nc file obsolete?  Are the differences from one version to another described anywhere?
7. 


In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
from clawpack.visclaw import geoplot
from importlib import reload
import geopandas as gpd
import folium
import pandas as pd

In [ ]:
import find_topo_source

## Some utility functions

In [ ]:
def read_tif(url):
    import rasterio
    with rasterio.open(url) as src:
        print(f"Coordinate Reference System (CRS): {src.crs}")
        print(f"Bounds: {src.bounds}")
        print(f"Number of bands: {src.count}")
        bounds = src.bounds
        print(f"Reading Z data array with shape {src.width}x{src.height}...")
        topotif = src.read()
    Z = flipud(topotif[0,:,:])
    x1,x2,y1,y2 = extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]
    dx = (x2-x1)/Z.shape[0]
    dy = (y2-y1)/Z.shape[1]
    print(f'dx = {dx:.7f} degrees = {dx*3600:.7f} arcseconds')
    print(f'dy = {dy:.7f} degrees = {dy*3600:.7f} arcseconds')
    
    x = arange(x1+0.5*dx, x2, dx)
    y = arange(y1+0.5*dy, y2, dy)
    
    topo = topotools.Topography()
    topo.set_xyZ(x,y,Z)
    return topo

In [ ]:
def check_tile(topo):
    dx = dy = 1/(9*3600)
    print(f'topo.Z has {len(topo.x)} points in x, {len(topo.y)} points in y')
    print(f'topo.x goes from {topo.x[0]} to {topo.x[-1]}')
    print(f'topo.x alignment: {mod((topo.x[0]+124.)/dx, 1):.3f} * dx away from integer longitudes')
    print(f'topo.y goes from {topo.y[0]} to {topo.y[-1]}')
    print(f'topo.y alignment: {mod((topo.y[0]-47.)/dy, 1):.3f} * dy away from integer latitudes')

## Select a set of tiles:

In [ ]:
eighth = 1/8  # half tile width, 0.0125 degrees
yrange = arange(46.5, 47.25+eighth, 0.25)
xrange = arange(-124.25, -123.5+eighth, 0.25)
name = 'NCEI tiles in SW Washington'
find_topo_source.make_combined_tile_kml(name, xrange, yrange)

In [ ]:
# print out tiles selected (north to south):
tile_names = []
for y in yrange[-1::-1]:
    for x in xrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        print(f'midpoints: {xm:.3f}, {ym:.3f}, NW corner:  {x:.3f}, {y:.3f}, tile: {tile_name}')
        tile_names.append(tile_name)

### map of tiles

In [ ]:
gdf = gpd.read_file('NCEI_tiles_in_SW_Washington.kml', driver='KML')
m = folium.Map(location=(46.75,-124), zoom_start=9, height=800, scrollWheelZoom=False)
folium.GeoJson(gdf).add_to(m)

for y in yrange[-1::-1]:
    for x in xrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        folium.Marker(
            location=[y-eighth,x+eighth],
            popup = f"<b>Tile:</b>\n {tile_name}",
            tooltip = "Click for info",
            #tooltip = f"<b>Tile:</b>\n {tile_name}",
            icon=folium.Icon(color="red", icon="info-sign") # Customize the marker's appearance
        ).add_to(m) 

for lat in arange(46.25, 47.3, 0.25):
    folium.PolyLine(
            [[lat,-124.5], [lat,-123.25]],
            color='black', weight=1, opacity=0.5
        ).add_to(m)
    # Add label at edge
    folium.Marker(
        [lat+0.03, -124.4],
        icon=folium.DivIcon(html=f'<div style="font-size: 12pt; color: gray;">{lat}°</div>')
        ).add_to(m)
m

## Versions of these tiles found on NCEI webpages:

In [ ]:
for tile_name in tile_names:
    print('\n-----------------------------------')
    tile_urls = find_topo_source.find_tile_url(tile_name, verbose=True)
    if len(tile_urls) == 0:
        print(f'{tile_name} not found')

## Compare n47x25_w124x00_2025v1 netCDF vs tiff versions

In [ ]:
url = 'https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/ncei19_n47x25_w124x00_2025v1.tif'
tile_n47x25_w124x00tif = read_tif(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/sowa_mhw_19_n47x25_w124x00_2025v1.nc'
tile_n47x25_w124x00nc = topotools.read_netcdf(url)

In [ ]:
tile = tile_n47x25_w124x00nc
name = 'tile_n47x25_w124x00nc'
dx = dy = 1/(9*3600)

print(f'\nTile: {name}:')
print(f'    {len(tile.x)} longitudes from {tile.x[0]:.6f} to {tile.x[-1]:.6f}')
print(f'    {len(tile.y)} latitudes  from {tile.y[0]:.6f} to {tile.y[-1]:.6f}')
print(f'    x alignment: {mod((tile.x[0]+124.)/dx, 1):.3f} * dx')
print(f'    y alignment: {mod((tile.y[0]-47.)/dy, 1):.3f} * dy')

tile = tile_n47x25_w124x00tif
name = 'tile_n47x25_w124x00tif'
print(f'\nTile: {name}:')
print(f'    {len(tile.x)} longitudes from {tile.x[0]:.6f} to {tile.x[-1]:.6f}')
print(f'    {len(tile.y)} latitudes  from {tile.y[0]:.6f} to {tile.y[-1]:.6f}')
print(f'    x alignment: {mod((tile.x[0]+124.)/dx, 1):.3f} * dx')
print(f'    y alignment: {mod((tile.y[0]-47.)/dy, 1):.3f} * dy')

In [ ]:
tile_n47x25_w124x00tif.crop(coarsen=18).plot(limits=(-20,200))

### Compare transects of data from two versions

We select a small sliver of this tile along the southern boundary, cutting across the Hoquiam River:

In [ ]:
extent_hoq = [-123.905, -123.881, 47-0.6*dy, 47+2.4*dy]
tile_nc = tile_n47x25_w124x00nc.crop(extent_hoq)
tile_tif = tile_n47x25_w124x00tif.crop(extent_hoq)

In [ ]:
tile_nc.Z.shape, tile_tif.Z.shape

In [ ]:
tile_nc.y-dy/2, tile_tif.y

In [ ]:
tile_nc.x[:4]-dx/2, tile_tif.x[:4]

In [ ]:
figure(figsize=(10,5))
j = 0
plot(tile_nc.x, tile_nc.Z[j,:], 'g', label=f'tile_nc at y={tile_nc.y[j]:.6f}')
plot(tile_tif.x, tile_tif.Z[j,:], 'r', label=f'tile_tif at y={tile_tif.y[j]:.6f}')
grid(True)
legend(loc='upper left', framealpha=1)

### Adjust the NAVD88 tif file data to MHW

`dz_mhw` chosen so that curves below align in flat regions.

In [ ]:
dz_mhw = -2.34
tile_tif_mhw = topotools.Topography()
tile_tif_mhw.set_xyZ(tile_tif.x, tile_tif.y, tile_tif.Z + dz_mhw)

In [ ]:
figure(figsize=(10,5))
j = 0

plot(tile_tif_mhw.x, tile_tif_mhw.Z[j,:], 'g', label=f'tile_tif_mhw at y={tile_tif_mhw.y[j]:.6f}')
plot(tile_nc.x, tile_nc.Z[j,:], 'r--', label=f'tile_nc at y={tile_nc.y[j]:.6f}')
plot(tile_tif_mhw.x, tile_tif_mhw.Z[j+1,:], 'b', label=f'tile_tif_mhw at y={tile_tif_mhw.y[j+1]:.6f}')
plot(tile_nc.x, tile_nc.Z[j+1,:], 'm--', label=f'tile_nc at y={tile_nc.y[j+1]:.6f}')
plot(tile_tif_mhw.x, tile_tif_mhw.Z[j+2,:], 'c', label=f'tile_tif_mhw at y={tile_tif_mhw.y[j+2]:.6f}')
grid(True)
legend(loc='lower right', framealpha=1)
xlim(-123.905, -123.892)
ylim(-4,4);

### Note offsets in elevations Z:

In the plot above, three rows of data from the `.tif` file (adjusted to MHW) are shown along with 2 rows of data from the `.nc` file, which are at intermediate latitudes due to the offset in `y` values.

Note that the dashed lines seem to interpolate the values from the solid lines, so presumably some averaging was used to get one set from the other.

In [ ]:
tile_tif.write('HOQ.txt',topo_type=1)

In [ ]:
ls -lh HOQ.txt

In [ ]:
!head -6 HOQ.txt

## Westport: Compare versions of n47x00_w124x25

Three versions were found on the NCEI websites, labelled `2018v1.nc`, `2025v1.nc` and `2025v2.tif`.
We also have a modified version from the Westport Maritime study that corrected the topo around the marina to include the seawalls better.

In [ ]:
url = 'https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/ncei19_n47x00_w124x25_2025v2.tif'
topo_2025v2 = read_tif(url)

In [ ]:
check_tile(topo_2025v2)

### Read netCDF version and observe the different offsets:

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/sowa_mhw_19_n47x00_w124x25_2025v1.nc'
topo_2025v1 = topotools.read_netcdf(url)

In [ ]:
check_tile(topo_2025v1)

In [ ]:
topo_2025v1.crop(coarsen=18).plot(limits=(-100,50))

## Approximate conversion from NAVD88 to MHW

The [web version of VDATUM](https://vdatum.noaa.gov/vdatumweb/vdatumweb?a=081402220260219) gives $-2.194$ m at Westport Marina, but fiddling with the comparison below gives a sligtly different value:

In [ ]:
dz_mhw = -2.3  # approximate correction NAVD88 to MHW
topo_2025v2_mhw = topotools.Topography()
topo_2025v2_mhw.set_xyZ(topo_2025v2.x, topo_2025v2.y, topo_2025v2.Z + dz_mhw)

### Create interpolating functions for extracting transects:

In [ ]:
topo_2025v1_fcn = topo_2025v1.make_function()
topo_2025v2_mhw_fcn = topo_2025v2_mhw.make_function()

## Compare seawalls in Westport Marina

In [ ]:
extent = [-124.12,-124.10, 46.903,46.915]
WPmarina1 = topo_2025v1.crop(extent)
WPmarina2 = topo_2025v2_mhw.crop(extent)

In [ ]:
from clawpack.visclaw import plottools
reload(plottools)
reload(topotools)

fig,axs = subplots(1, 2, sharey=True, figsize=(10,5))
WPmarina1.plot(axes=axs[0], add_colorbar=False)
axs[0].set_title('WPmarina_2025v1')
WPmarina2.plot(axes=axs[1], add_colorbar=False)
axs[1].set_title('WPmarina_2025v2')
tight_layout()

In [ ]:
fname = 'A519_n46x99_w124x24_2021v1_warp_Pat_Wide_Seawalls.nc'
topodir = '/Users/rjl/topo/wa_coast_mhw_2020/deliverables'
topo_2021_seawalls = topotools.read_netcdf(f'{topodir}/{fname}')

In [ ]:
check_tile(topo_2021_seawalls)

In [ ]:
topo_2021_seawalls_fcn = topo_2021_seawalls.make_function()

In [ ]:
y0 = 47.03
xtrans = linspace(-124.115, -124.105, 1000)
ytrans = linspace(46.904, 46.910, 1000)

figure(figsize=(6,5))
WPmarina1.plot(limits=(-10,10), cb_kwargs={'extend':'both'})
plot(xtrans,ytrans,'k')
title('Transect')

# transects plots:
fig,ax = subplots(figsize=(10,5))
plot(xtrans, topo_2025v2_mhw_fcn(xtrans,ytrans), 'r', label='topo_2025v2')
plot(xtrans, topo_2025v1_fcn(xtrans,ytrans), 'g', label='topo_2025v1')
plot(xtrans, topo_2021_seawalls_fcn(xtrans,ytrans), 'k', linewidth=0.7, label='topo_2021_seawalls')
grid(True)
#ylim(-3,10.)
xlim(-124.114, -124.106)
ylim(-2,4)
ticklabel_format(useOffset=False)
xticks(rotation=20)
legend(framealpha=1)
title(f'Transect through Westport Marina');

## Compare versions of n47x50_w124x50

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/tiled_19as/ncei19_n47x50_w0124x50_2018v1.nc'
topo1 = topotools.read_netcdf(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/ncei19_n47x50_w0124x50_2020.nc'
topo1a = topotools.read_netcdf(url)

In [ ]:
print(topo1.x[:3], topo1.x[-3:])
print(topo1.y[:3], topo1.y[-3:])

In [ ]:
print(topo1a.x[:3], topo1a.x[-3:])
print(topo1a.y[:3], topo1a.y[-3:])

In [ ]:
dx = dy = 1/(9*3600)
print(f'topo1.x alignment: {mod((topo1.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo1.y alignment: {mod((topo1.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'topo1a.x alignment: {mod((topo1a.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo1a.y alignment: {mod((topo1a.y[0]-47.)/dy, 1):.3f} * dx')

## Compare versions of n47x25_w124x25

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/tiled_19as/ncei19_n47x25_w0124x25_2018v1.nc'
topo2 = topotools.read_netcdf(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/ncei19_n47x25_w0124x25_2020.nc'
topo2a = topotools.read_netcdf(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/sowa_mhw_19_n47x25_w124x25_2025v1.nc'
topo2b = topotools.read_netcdf(url)

In [ ]:
print(f'topo2.x alignment: {mod((topo2.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo2.y alignment: {mod((topo2.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'topo2a.x alignment: {mod((topo2a.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo2a.y alignment: {mod((topo2a.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'topo2b.x alignment: {mod((topo2b.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo2b.y alignment: {mod((topo2b.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
topo2f = topo2.make_function()
topo2af = topo2a.make_function()
topo2bf = topo2b.make_function()

In [ ]:
y0 = 47.03
xtrans = arange(-124.2, -124, dx/5)
ytrans = y0 * ones(xtrans.shape)
dz_mhw = 2.3  # approximate correction NAVD88 to MHW
fig,ax = subplots(figsize=(10,5))
plot(xtrans, topo2f(xtrans,ytrans) - dz_mhw, 'g', label='topo2 - 2018 adjusted to MHW')
plot(xtrans, topo2af(xtrans,ytrans), 'r', label='topo2a - 2020')
plot(xtrans, topo2bf(xtrans,ytrans), 'b', label='topo2b - 2025')
grid(True)
#ylim(-3,10.)
xlim(-124.18, -124.1)
ylim(-3,6)
legend(framealpha=1)
title(f'Transect through Ocean Shores at y = {y0:.5f}');

In [ ]:
dz = topo2f(xtrans,ytrans) - topo2af(xtrans,ytrans)
dz.min(), dz.max()